# RT-DETR Pretraining with SHIFT-Discrete Dataset

## Check GPU Availability

In [1]:
!nvidia-smi

Sun Sep 21 11:24:45 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:04:00.0 Off |                    0 |
| N/A   67C    P0              78W / 250W |   7512MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# Set CUDA Device Number
DEVICE_NUM = 6

from os import environ
environ["CUDA_VISIBLE_DEVICES"] = str(DEVICE_NUM)
environ["CUDA_VISIBLE_DEVICES"]

'6'

## Imports

In [3]:
import os
os.chdir("/workspace/ptta") # os.chdir("/home/ubuntu/test-time-adapters")

In [4]:
from os import path
import math

import torch
from torch.utils.data import DataLoader

from ttadapters.datasets import BaseDataset, DatasetHolder, DataLoaderHolder
from ttadapters.datasets import (
    SHIFTDataset,
    SHIFTClearDatasetForObjectDetection,
    SHIFTCorruptedDatasetForObjectDetection,
    SHIFTDiscreteSubsetForObjectDetection
)
from ttadapters import datasets

from ttadapters.models.rcnn import FasterRCNNForObjectDetection, SwinRCNNForObjectDetection

from supervision.metrics.mean_average_precision import MeanAveragePrecision
from supervision.detection.core import Detections

from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"INFO: Using device - {device}")

INFO: Using device - cuda


In [6]:
PROJECT_NAME = "detectron_test"
RUN_NAME = "Faster-RCNN_R50"

## Define Dataset

In [7]:
DATA_ROOT = path.join(".", "data")

## DataLoader

In [8]:
from detectron2.structures import ImageList

def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    return dict(
        pixel_values=ImageList.from_tensors(images, size_divisibility=32),
        labels=[dict(
            class_labels=item['boxes2d_classes'].long(),
            boxes=item["boxes2d"].float()
        ) for item in targets]
    )

In [9]:
from detectron2.structures import Boxes, Instances
from torchvision.tv_tensors import Image, BoundingBoxes

def collate_fn(batch: list[Image, BoundingBoxes]):
    batched_inputs = []
    for image, metadata in batch:
        original_height, original_width = image.shape[-2:]
        instances = Instances(image_size=(original_height, original_width))
        instances.gt_boxes = Boxes(metadata["boxes2d"])  # xyxy
        instances.gt_classes = metadata["boxes2d_classes"]
        batched_inputs.append({
            "image": image,
            "instances": instances,
            "height": original_height,
            "width": original_width
        })
    return batched_inputs

## Load Model

In [10]:
USE_SWIN_T_BACKBONE = False

In [11]:
if USE_SWIN_T_BACKBONE:
    model = SwinRCNNForObjectDetection(dataset=SHIFTDataset)
else:
    model = FasterRCNNForObjectDetection(dataset=SHIFTDataset)

model.load_from(model.Weights.NATUREYOO, weight_key="model")
model.to(device)

FasterRCNNForObjectDetection(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2): Sequential(
        (0): Bo

### Direct Test

In [12]:
def task_to_subset_types(task: str):
    T = SHIFTDiscreteSubsetForObjectDetection.SubsetType

    # weather
    if task == "cloudy":
        return T.CLOUDY_DAYTIME
    if task == "overcast":
        return T.OVERCAST_DAYTIME
    if task == "rainy":
        return T.RAINY_DAYTIME
    if task == "foggy":
        return T.FOGGY_DAYTIME

    # time
    if task == "night":
        return T.CLEAR_NIGHT
    if task in {"dawn", "dawn/dusk"}:
        return T.CLEAR_DAWN
    if task == "clear":
        return T.CLEAR_DAYTIME
    
    # simple
    if task == "normal":
        return T.NORMAL
    if task == "corrupted":
        return T.CORRUPTED

    raise ValueError(f"Unknown task: {task}")

In [13]:
# dataset
class SHIFTCorruptedDatasetForObjectDetection(SHIFTDiscreteSubsetForObjectDetection):
    def __init__(
            self, root: str, force_download: bool = False,
            train: bool = True, valid: bool = False,
            transform= None, target_transform = None, transforms = None,
            task = "clear"
    ):
        super().__init__(
            root=root, force_download=force_download,
            train=train, valid=valid, subset_type=task_to_subset_types(task),
            transform=transform, target_transform=target_transform, transforms=transforms
        )

In [14]:
import time
import gc

def evaluate_for(self, loader, loader_length, threshold=0.0, dtype=torch.float32, device=torch.device("cuda")):
    torch.cuda.empty_cache()
    gc.collect()

    self.eval()

    map_metric = MeanAveragePrecision()
    predictions_list = []
    targets_list = []
    collapse_time = 0

    with torch.inference_mode():
        for batch in tqdm(loader, total=loader_length, desc="Evaluation"):
            with torch.autocast(device_type=device.type, dtype=dtype):
                start = time.time()
                outputs = self(batch)
                collapse_time += time.time() - start

            for i, (output, input_data) in enumerate(zip(outputs, batch)):
                instances = output['instances']
                mask = instances.scores > threshold

                pred_detection = Detections(
                    xyxy=instances.pred_boxes.tensor[mask].detach().cpu().numpy(),
                    class_id=instances.pred_classes[mask].detach().cpu().numpy(),
                    confidence=instances.scores[mask].detach().cpu().numpy()
                )
                gt_instances = input_data['instances']
                target_detection = Detections(
                    xyxy=gt_instances.gt_boxes.tensor.detach().cpu().numpy(),
                    class_id=gt_instances.gt_classes.detach().cpu().numpy()
                )

                predictions_list.append(pred_detection)
                targets_list.append(target_detection)

        map_metric.update(predictions=predictions_list, targets=targets_list)
        m_ap = map_metric.compute()

        per_class_map = {
            f"{CLASSES[idx]}_mAP@0.50:0.95": m_ap.ap_per_class[idx].mean().item()
            for idx in m_ap.matched_classes
        }
        performances = {
            "collapse_time": collapse_time,
            "fps": loader_length / collapse_time
        }

        return {
            "mAP@0.50:0.95": m_ap.map50_95.item(),
            "mAP@0.50": m_ap.map50.item(),
            "mAP@0.75": m_ap.map75.item(),
            **per_class_map,
            **performances
        }

In [15]:
from ttadapters.methods.other_method import utils

## Direct Method

In [16]:
# # direct_method
# for task in ["cloudy", "overcast", "foggy", "rainy", "dawn", "night", "clear"]:
#     dataset=SHIFTCorruptedDatasetForObjectDetection(
#         root=DATA_ROOT, valid=True,
#         transform=datasets.detectron_image_transform,
#         transforms=datasets.default_valid_transforms,
#         task=task
#     )
#     print(f"start {task}")
#     CLASSES = dataset
#     NUM_CLASSES = len(CLASSES)
    
#     dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
#     dataloader.valid_len = math.ceil(len(dataset)/4)
#     result = evaluate_for(model, dataloader, dataloader.valid_len)
#     print(result)

## ActMAD

In [17]:
import torch.optim as optim
import torch.nn as nn
from pathlib import Path
from detectron2.layers import FrozenBatchNorm2d
from detectron2.utils.events import EventStorage

In [18]:
def extract_activation_alignment(model, method, data_root, batch_size=16):
    dataset = SHIFTClearDatasetForObjectDetection(
        root=data_root, train=True,
        transform=datasets.detectron_image_transform,
        transforms=datasets.default_train_transforms
    )

    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    loader.train_len = math.ceil(len(dataset)/batch_size)

    # model unfreeze
    for k, v in model.named_parameters():
        v.requires_grad = True

    chosen_bn_info = []
    if method == "actmad": 
        for name, m in model.named_modules():
            if isinstance(m, (FrozenBatchNorm2d)):
                chosen_bn_info.append((name, m))

    # chosen_bn_layers
    """
    Since high-level representations are more sensitive to domain shift,
    only the later BN layers are selected. 
    The cutoff point is determined empirically.
    """
    cutoff = len(chosen_bn_info) // 2
    chosen_bn_info = chosen_bn_info[cutoff:]
    chosen_bn_layers = [module for name, module in chosen_bn_info]
    layer_names = [name for name, module in chosen_bn_info]

    n_chosen_layers = len(chosen_bn_layers)

    save_outputs = [utils.SaveOutput() for _ in range(n_chosen_layers)]

    clean_mean_act_list = [utils.AverageMeter() for _ in range(n_chosen_layers)]
    clean_var_act_list = [utils.AverageMeter() for _ in range(n_chosen_layers)]

    clean_mean_list_final = []
    clean_var_list_final = []
    
    # extract the activation alignment in train dataset
    print("Start extracting BN statistics from the training dataset")
    
    with torch.no_grad():
        for batch in tqdm(loader, total=loader.train_len, desc="Evaluation"):
            model.eval()
            hook_list = [chosen_bn_layers[i].register_forward_hook(save_outputs[i]) for i in range(n_chosen_layers)]
            _ = model(batch)

            for i in range(n_chosen_layers):
                clean_mean_act_list[i].update(save_outputs[i].get_out_mean())  # compute mean from clean data
                clean_var_act_list[i].update(save_outputs[i].get_out_var())  # compute variane from clean data

                save_outputs[i].clear()
                hook_list[i].remove()

        for i in range(n_chosen_layers):
            clean_mean_list_final.append(clean_mean_act_list[i].avg)  # [C, H, W]
            clean_var_list_final.append(clean_var_act_list[i].avg)  # [C, H, W]

        return clean_mean_list_final, clean_var_list_final, layer_names

In [19]:
# actmad | extract clear data bn

# hyperparameter 
CLEAN_BN_EXTRACT_BATCH = 8
stats_save_path = Path("/workspace/ptta/ttadapters/methods/other_method") / f"actmad_clean_statistics_faster_rcnn.pt"

statistics = {}

# 저장된 statistics가 있는지 확인
if stats_save_path.exists():
    print(f"Loading saved ActMAD statistics from {stats_save_path}")
    saved_stats = torch.load(stats_save_path)
    statistics["clean_mean_list_final"] = saved_stats["clean_mean_list_final"]
    statistics["clean_var_list_final"] = saved_stats["clean_var_list_final"]
    statistics["layer_names"] = saved_stats["layer_names"]
else:
    print("Extracting ActMAD statistics from clean data...")
    (
        statistics["clean_mean_list_final"],
        statistics["clean_var_list_final"],
        statistics["layer_names"]
    ) = extract_activation_alignment(
        model=model, method="actmad",
        data_root=DATA_ROOT, 
        batch_size=CLEAN_BN_EXTRACT_BATCH
        )

    # Statistics만 저장 (chosen_bn_layers는 저장하지 않음)
    print(f"Saving ActMAD statistics to {stats_save_path}")
    torch.save({
        "clean_mean_list_final": statistics["clean_mean_list_final"],
        "clean_var_list_final": statistics["clean_var_list_final"],
        "layer_names": statistics["layer_names"]
    }, stats_save_path)

Loading saved ActMAD statistics from /workspace/ptta/ttadapters/methods/other_method/actmad_clean_statistics_faster_rcnn.pt


In [ ]:
clean_mean_list_final = statistics["clean_mean_list_final"]
clean_var_list_final = statistics["clean_var_list_final"]
layer_names = statistics["layer_names"]

current_bn_dict = {name: module for name, module in model.named_modules()
                    if isinstance(module, FrozenBatchNorm2d)}

chosen_bn_layers = []
for layer_name in layer_names:
    if layer_name in current_bn_dict:
        chosen_bn_layers.append(current_bn_dict[layer_name])
    else:
        print(f"Warning: Layer {layer_name} not found!")

optimizer = optim.SGD(
                model.parameters(),
                lr=0.0001,  
            )
# Unfreeze model parameters for ActMAD
for k, v in model.named_parameters():
    v.requires_grad = True

for task in ["cloudy", "overcast", "foggy", "rainy", "dawn", "night", "clear"]:
    map_metric = MeanAveragePrecision()
    predictions_list = []
    targets_list = []
    threshold = 0.0

    # data load
    dataset=SHIFTCorruptedDatasetForObjectDetection(
        root=DATA_ROOT, valid=True,
        transform=datasets.detectron_image_transform,
        transforms=datasets.default_valid_transforms,
        task=task
    )
    print(f"start {task}")
    CLASSES = dataset
    NUM_CLASSES = len(CLASSES)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
    dataloader.valid_len = math.ceil(len(dataset)/4)

    # Unfreeze model parameters for ActMAD
    for param in model.parameters():
        param.requires_grad = True
    
    n_chosen_layers = len(chosen_bn_layers)

    l1_loss = nn.L1Loss(reduction="mean")

    for batch in tqdm(dataloader, total=dataloader.valid_len, desc="Evaluation"):
        model.eval()
        # for m in model.modules():
        #     if isinstance(m, (FrozenBatchNorm2d)):
        #         m.eval()
        optimizer.zero_grad()
        save_outputs_tta = [utils.SaveOutput() for _ in range(n_chosen_layers)]

        hook_list_tta = [chosen_bn_layers[x].register_forward_hook(save_outputs_tta[x])
                        for x in range(n_chosen_layers)]
        
        # forward pass
        outputs = model(batch)

        # Extract current batch statistics
        batch_mean_tta = [save_outputs_tta[x].get_out_mean() for x in range(n_chosen_layers)]
        batch_var_tta = [save_outputs_tta[x].get_out_var() for x in range(n_chosen_layers)]

        # Compute ActMAD loss
        loss_mean = torch.tensor(0, requires_grad=True, dtype=torch.float).float().to(device)
        loss_var = torch.tensor(0, requires_grad=True, dtype=torch.float).float().to(device)

        for i in range(n_chosen_layers):
            loss_mean += l1_loss(batch_mean_tta[i].to(device), clean_mean_list_final[i].to(device))
            loss_var += l1_loss(batch_var_tta[i].to(device), clean_var_list_final[i].to(device))
            
        loss =  loss_mean +  loss_var

        # Backward and update
        loss.backward()
        optimizer.step()

        # Clean up hooks 
        for z in range(n_chosen_layers):
            save_outputs_tta[z].clear()
            hook_list_tta[z].remove()
        
        for i, (output, input_data) in enumerate(zip(outputs, batch)):
                instances = output['instances']
                mask = instances.scores > threshold

                pred_detection = Detections(
                    xyxy=instances.pred_boxes.tensor[mask].detach().cpu().numpy(),
                    class_id=instances.pred_classes[mask].detach().cpu().numpy(),
                    confidence=instances.scores[mask].detach().cpu().numpy()
                )
                gt_instances = input_data['instances']
                target_detection = Detections(
                    xyxy=gt_instances.gt_boxes.tensor.detach().cpu().numpy(),
                    class_id=gt_instances.gt_classes.detach().cpu().numpy()
                )

                predictions_list.append(pred_detection)
                targets_list.append(target_detection)

    map_metric.update(predictions=predictions_list, targets=targets_list)
    print(f"start {task} mAP computation")
    m_ap = map_metric.compute()

    per_class_map = {
        f"{CLASSES[idx]}_mAP@0.50:0.95": m_ap.ap_per_class[idx].mean().item()
        for idx in m_ap.matched_classes
    }

    print({
        "mAP@0.50:0.95": m_ap.map50_95.item(),
        "mAP@0.50": m_ap.map50.item(),
        "mAP@0.75": m_ap.map75.item(),
        **per_class_map,
    })


INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...


[09/21/2025 11:25:01] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7f83588e1d30>
[09/21/2025 11:25:01] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/cloudy_daytime/discrete/images/val/front/det_2d.json' ...


INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[09/21/2025 11:25:01] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/cloudy_daytime/discrete/images/val/front/det_2d.json' Done.
[09/21/2025 11:25:03] SHIFT DevKit - INFO - Loading annotation takes 2.43 seconds.


Batch 0:

Item                 Shape                               Min        Max       
--------------------------------------------------------------------------------
original_hw          [tensor([800]), tensor([1280])]
input_hw             [tensor([800]), tensor([1280])]
frame_ids            torch.Size([1])                           0.00       0.00
name                 ['00000000_img_front.jpg']
videoName            ['075f-be61']
intrinsics           torch.Size([1, 3, 3])                     0.00     640.00
extrinsics           torch.Size([1, 4, 4])                    -0.74     159.04
boxes2d              torch.Size([1, 9, 4])                     0.00    1044.00
boxes2d_classes      torch.Size([1, 9])                        0.00       2.00
boxes2d_track_ids    torch.Size([1, 9])                        0.00       8.00
images               torch.Size([1, 3, 800, 1280])             0.00     255.00

Video name: 075f-be61
Sample indices within a video: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,

...

[09/21/2025 11:40:31] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/overcast_daytime/discrete/images/val/front/det_2d.json' Done.
[09/21/2025 11:40:33] SHIFT DevKit - INFO - Loading annotation takes 1.57 seconds.


Batch 0:

Item                 Shape                               Min        Max       
--------------------------------------------------------------------------------
original_hw          [tensor([800]), tensor([1280])]
input_hw             [tensor([800]), tensor([1280])]
frame_ids            torch.Size([1])                           0.00       0.00
name                 ['00000000_img_front.jpg']
videoName            ['0aee-69fd']
intrinsics           torch.Size([1, 3, 3])                     0.00     640.00
extrinsics           torch.Size([1, 4, 4])                    -5.76     162.09
boxes2d              torch.Size([1, 5, 4])                   255.00     881.00
boxes2d_classes      torch.Size([1, 5])                        1.00       5.00
boxes2d_track_ids    torch.Size([1, 5])                        0.00       4.00
images               torch.Size([1, 3, 800, 1280])             0.00     255.00

Video name: 0aee-69fd
Sample indices within a video: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,

Evaluation:   0%|          | 0/400 [00:00<?, ?it/s]

start overcast mAP computation


KeyboardInterrupt: 